In [10]:
import pandas as pd
import numpy as np
import nltk
import math
import time
import operator
import string
import csv
from sklearn.cluster import KMeans
from math import pi
from sklearn.metrics import mean_squared_error
from nltk.corpus import wordnet, stopwords, sentiwordnet as swn
from nltk.tokenize import sent_tokenize, word_tokenize, WhitespaceTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
f = open('train_data.csv','r', errors='ignore')
train_data = pd.read_csv(f,
                header=0, sep=',',
                parse_dates=['date', 'release_date', 'update_date'],
                infer_datetime_format=True)
length_train = len(train_data.review_text)

f1 = open('test_data.csv','r', errors='ignore')
test_data = pd.read_csv(f1,
                header=0, sep=',',
                parse_dates=['date', 'release_date', 'update_date'],
                infer_datetime_format=True)
train_data.head()


,asin,date,downvotes,review_comments,review_id,review_text,review_type,reviewer,star_rating,upvotes,...,in_app_purchase,mas_rating,min_os_version,num_permissions,num_screenshots,price,release_date,size_MB,title,update_date
670000,B00992CF6W,2016-03-22,NaN,0,RXKYJIBS93L4T,I Love minecraft it has multiplayer maps and c...,Verified Purchase,AB68CC4T9SS5S,5,NaN,...,1,Guidance Suggested,Varies,10,9,699.0,2012-09-13,36.3,Minecraft - Pocket Edition,2016-08-10
670001,B00992CF6W,2015-08-11,NaN,0,RXL1DJN1UJ32R,FUN APP,Verified Purchase,A9J6SKF2DVWCZ,5,NaN,...,1,Guidance Suggested,Varies,10,9,699.0,2012-09-13,36.3,Minecraft - Pocket Edition,2016-08-10
670002,B00992CF6W,2015-05-24,NaN,0,RXL2DUNRCA7LA,Needs updates. But it is the best block deal o...,Verified Purchase,A4QF3KS9MMUQX,5,NaN,...,1,Guidance Suggested,Varies,10,9,699.0,2012-09-13,36.3,Minecraft - Pocket Edition,2016-08-10
670003,B00992CF6W,2015-11-15,NaN,0,RXLCRHXV4TAGO,This game is so cool. The only bad part is wai...,Verified Purchase,A14VAA05K1S9D5,5,NaN,...,1,Guidance Suggested,Varies,10,9,699.0,2012-09-13,36.3,Minecraft - Pocket Edition,2016-08-10
670004,B00992CF6W,2015-12-01,NaN,0,RXLG25CTW1B4T,Agufdgdbdbsbs Bibb,Verified Purchase,AEU76SYXRWHYO,5,NaN,...,1,Guidance Suggested,Varies,10,9,699.0,2012-09-13,36.3,Minecraft - Pocket Edition,2016-08-10
670005,B00992CF6W,2015-02-20,NaN,0,RXLM2MKLWOWF9,I have played mine craft before and I loved it...,Verified Purchase,A2UEHRDWTAR91W,5,NaN,...,1,Guidance Suggested,Varies,10,9,699.0,2012-09-13,36.3,Minecraft - Pocket Edition,2016-08-10
670006,B00992CF6W,2014-09-18,NaN,0,RXLN4WI42NX5E,My son lovee the game and plays it with his fr...,Verified Purchase,A3HJS90QJ4YZ0T,4,NaN,...,1,Guidance Suggested,Varies,10,9,699.0,2012-09-13,36.3,Minecraft - Pocket Edition,2016-08-10
670007,B00992CF6W,2013-02-13,NaN,0,RXLT29GBP7WQD,My 10 year old says its one of the best games ...,Verified Purchase,A51XGLZ3NN12W,5,NaN,...,1,Guidance Suggested,Varies,10,9,699.0,2012-09-13,36.3,Minecraft - Pocket Edition,2016-08-10
670008,B00992CF6W,2014-09-02,NaN,0,RXLUW152YYGQ7,Bought this for my 7-year-old granddaughter. ...,Verified Purchase,A3Q4AH81G6DWIU,5,NaN,...,1,Guidance Suggested,Varies,10,9,699.0,2012-09-13,36.3,Minecraft - Pocket Edition,2016-08-10
670009,B00992CF6W,2016-02-27,NaN,0,RXM23JFP564C5,THE BEST GAME IN THE WORLD!!!!!!!!,Verified Purchase,A6NTAPXMEP2H9,5,NaN,...,1,Guidance Suggested,Varies,10,9,699.0,2012-09-13,36.3,Minecraft - Pocket Edition,2016-08-10


In [12]:
train_review = pd.Series.tolist(train_data.review_text)
train_rating = pd.Series.tolist(train_data.star_rating)
train_model = [None]*5
for i in range(5):
    train_model[i] = [str(tmp) for tmp, score in zip(train_review, train_rating) if score == i+1]

In [13]:
most_word = [None]*5
feature = [None]*5
feature_learn = [None]*5
test_model = [None]*5
test_review = pd.Series.tolist(test_data.review_text)
test_review = [str(tmp) for tmp in test_review]
#train_review = [str(tmp) for tmp in train_review]
voc_model = TfidfVectorizer(decode_error='ignore', ngram_range=(1,2), max_df=0.8, min_df=0.01, stop_words='english')
#voc_tf_model = CountVectorizer(decode_error='ignore', ngram_range=(1,2), stop_words='english')
for i in range(5):
    feature[i] = voc_model.fit_transform(train_model[i])
    #most_word[i] = voc_model.get_feature_names()
    feature_learn[i] = voc_model.transform(train_model[i])
    test_model[i] = voc_model.transform(test_review)

In [50]:
# for i in range(5):
#     for words in most_word[i]:
#         syns = wordnet.synsets(words)
#         word_score_pos = 0
#         word_score_neg = 0
#         for t in range(len(syns)):
#             if word_score_pos < swn.senti_synset(syns[t].name()).pos_score():
#                 word_score_pos = swn.senti_synset(syns[t].name()).pos_score()
#             if word_score_neg < swn.senti_synset(syns[t].name()).neg_score():
#                 word_score_neg = swn.senti_synset(syns[t].name()).neg_score()
#         if word_score_pos == 0 and word_score_neg == 0:
#             most_word[i].remove(words)

In [14]:
kmeans = KMeans(n_clusters = 1, init='k-means++')
centroids = [None]*5
for i in range(5):
    kmeans.fit(feature_learn[i])
    centroids[i] = kmeans.cluster_centers_

In [17]:
distance = np.zeros((5,100000))
for i in range(5):
    for j in range(100000):
        distance[i][j] = mean_squared_error(centroids[i], test_model[i][j].toarray())

In [18]:
prob = np.zeros((100000,5))
distance_t = np.transpose(distance)
for i in range(100000):
    for j in range(5):
        prob[i][j] = distance_t[i][j]/sum(distance_t[i])

In [452]:
prob1 = prob.astype('str')
review_id = pd.Series.tolist(test_data.review_id)
review_id = np.asarray([[str(tmp)] for tmp in review_id])

In [453]:
result = np.append(review_id, prob1, 1)
title = np.asarray([['review_id','1-star','2-star','3-star','4-star','5-star']])
result = np.append(title,result,0)
#result = [np.array(tmp) for tmp in result]
result = result.tolist()
result = np.asarray(result)
np.savetxt("result.csv",result,fmt='%s,%s,%s,%s,%s,%s')